#
 module name: mdll_cmp_weighted
 - read "ML_Classifier_Comparision_091423.xlsx

 - drop Level 1 if one of level 2 doesn't have the predicted value
 - use groupby('Level 1') to get  the weighted difference and created the DF df_grp
   - The weighted difference = the sum of ( accuracy difference * GTIN cnt)/ (sum of GTIN cnt)
      - GTIN  cnt for each Level 2 is from syndigo_gtin_level_257K_081523.csv
 - rename the col nmae None to 'weighted difference'

In [1]:
import pandas as pd
pd.set_option("display.max_rows", None)
from openpyxl import load_workbook
path = 'C:\\users\\iny2819\\kroger\\Data\\'   

In [2]:
f_name = path + "ML_Classifier_Comparision_091423.xlsx"
df = pd.read_excel(f_name, sheet_name = 'Cmp')
df.head()

,Level 1,Level 2,LR test Accuracy,MLP_smote test accuracy
0,Agricultural Equipment,Tires,NaN,NaN
1,Apparel,Clothing,0.999832,0.999767
2,Apparel,Footwear,0.999078,0.998904
3,Apparel,Personal Accessories,0.956785,0.964204
4,Apparel,Safety Apparel,0.815787,0.854019


In [3]:
df.shape

(231, 4)

In [4]:

# get GTIN from Syndigo file 
df_gtin = pd.read_csv(path + "syndigo_gtin_level_257K_081523.csv",  dtype='unicode')
df_gtin.head(2)

,GTIN,Level 1,Level 2,Level 3,Level 4,Level 5,Level 6,Level 7,Level 8,Level 9
0,86003018082,Beer / Wine / Spirits,Wine,Still,NaN,NaN,NaN,NaN,NaN,NaN
1,33900002157,Food / Beverages,Meat / Poultry / Seafood / Meat Substitutes (P...,Prepared / Processed Animal Products,Sandwich Meat,NaN,NaN,NaN,NaN,NaN


In [5]:
# got level1 and level 2 count information and merged with predicted result
dfs_gtin =df_gtin.groupby('Level 1', as_index = False)['Level 2'].value_counts(dropna = False)
dfs_gtin.rename(columns = {'count' : 'GTIN cnt' }, inplace = True  )
df = df.merge(dfs_gtin, on=['Level 1','Level 2'], how='left')
print(f' df.shape before dropna = {df.shape}')

 df.shape before dropna = (231, 5)


In [6]:
df_bkup = df.copy()

In [7]:
# drop Level 1 if one of level 2 doesn't have the predicted value 
df1 = df.groupby('Level 1', as_index = False).filter(lambda df: df.notnull().values.all())
df1.shape

(29, 5)

df.groupby('Level 1').apply(lambda x: ((x['LR test Accuracy'] - x['MLP_smote test  accuracy']) *  x['GTIN cnt']).sum())

In [9]:
df_grp = df1.groupby('Level 1', as_index = False).apply(lambda x: ( ((x['LR test Accuracy'] - x['MLP_smote test  accuracy']) *  x['GTIN cnt']).sum())/x['GTIN cnt'].sum())

In [10]:
df_grp

,Level 1,None
0,Apparel,-0.003487
1,Food / Beverages,0.002015
2,Gardening & Outdoors,-0.000265
3,Health & Beauty,0.000533
4,Heating / Ventilation / Air Conditioning,0.001383
5,Kitchen & Bathroom,-0.001000
6,Livestock & Pet Supplies,-0.000178


In [11]:
df_grp.rename(columns = {None : "weighted difference"}, inplace  = True)

In [12]:
df_grp

,Level 1,weighted difference
0,Apparel,-0.003487
1,Food / Beverages,0.002015
2,Gardening & Outdoors,-0.000265
3,Health & Beauty,0.000533
4,Heating / Ventilation / Air Conditioning,0.001383
5,Kitchen & Bathroom,-0.001000
6,Livestock & Pet Supplies,-0.000178


In [14]:
from datetime import date
dte = date.today().strftime('%m%d%y')
excel_file = path + "LR_MLPSMOTE weighted differences_"+dte + '.xlsx'
from pandas import ExcelWriter
from pandas import ExcelFile
writer = pd.ExcelWriter(excel_file)
df_grp.to_excel(writer,'LR_MLPSMOTE weighted difference')  
df.to_excel(writer,'LR_MLPSMOTE MDL CMP GTIN CNT')
df1.to_excel(writer,'LR_MLPSMOTE MDL CMP Dropna') 
     
writer.close()

In [ ]:
STOP

In [ ]:
# try to write the worksheet to the existing file, but not working.
#df = pd.read_excel('existing_file.xlsx', sheet_name='Sheet1')
# Load the existing Excel file using openpyxl
book = load_workbook(f_name)

# Create a new worksheet
#new_sheet = book.create_sheet(['LR_MLPSMOTE weighted difference', 'cmp_dropna'])
new_sheet = book.create_sheet('LR_MLPSMOTE weighted difference')

# Save the changes
book.save(f_name)
# Write data to the new worksheet
df_grp.to_excel(f_name, sheet_name='LR_MLPSMOTE weighted difference', index=False)

In [ ]:
len('LR_MLPSMOTE weighted difference')